In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import random

df = "/home/maksym/MatMod4/ita-eng/ita.txt"

english_txt = []
italian_txt = []

english_ch = set()
italian_ch = set()

with open(df, "r", encoding = "utf-8") as f:
    lines = f.read().split("\n")
print("#------------------------------0")
for line in lines[: min(20000, len(lines) - 1)]:
    english_text, italian_text, attribution = line.split("\t")

    italian_text = "\t" + italian_text + "\n"
    english_txt.append(english_text)
    italian_txt.append(italian_text)
    for ch in english_text:
        if ch not in english_ch:
            english_ch.add(ch)   
    for ch in italian_text:
        if ch not in italian_ch:
            italian_ch.add(ch)

num_iterations = 5


print("#------------------------------1")

#Sorting
#print('aasaa')

for _ in range(num_iterations):
    random_index = random.randint(0, len(english_txt) - 1)
    
    print(f"English: {english_txt[random_index]}")
    print(f"Italian: {italian_txt[random_index]}")
    #print("-" * 20)
max_enc_en_len = max([len(en_buf) for en_buf in english_txt])
max_dec_it_len = max([len(it_buf) for it_buf in italian_txt])
#print("-" * 20)
english_ch = sorted(list(english_ch))
italian_ch = sorted(list(italian_ch))


#target_ukr_token_index
print("#------------------------------2")
input_en_token_index = {ch: i for i, ch in enumerate(english_ch)}
tit_tindex = {ch: i for i, ch in enumerate(italian_ch)}
#encoder_input_data
#decoder_input_data
#decoder_target_data
enc_indata = np.zeros((len(english_txt), max_enc_en_len, len(english_ch)), dtype="float32")
dec_indata = np.zeros((len(english_txt), max_dec_it_len, len(italian_ch)), dtype="float32")
dec_tdata = np.zeros((len(english_txt), max_dec_it_len, len(italian_ch)), dtype="float32")
 
for i, (en, it) in enumerate(zip(english_txt, italian_txt)):
    for t, ch in enumerate(en):
        enc_indata[i, t, input_en_token_index[ch]] = 1.0
    enc_indata[i, t + 1:, input_en_token_index[" "]] = 1.0
    for t, ch in enumerate(it):
        dec_indata[i, t, tit_tindex[ch]] = 1.0
        if t > 0:
            dec_tdata[i, t - 1, tit_tindex[ch]] = 1.0   
    dec_indata[i, t + 1:, tit_tindex[" "]] = 1.0
    dec_tdata[i, t:, tit_tindex[" "]] = 1.0
#encoder_inputs
#encoder_outputs
enc_inputs = keras.Input(shape=(None, len(english_ch)))
encoder = keras.layers.LSTM(256, return_state=True)
enc_outputs, state_h, state_c = encoder(enc_inputs)
#------------------------------
print("#------------------------------3")
#encoder_states
#decoder_inputs
enc_st = [state_h, state_c]
dec_in = keras.Input(shape=(None, len(italian_ch)))
decoder_lstm = keras.layers.LSTM(256, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_in, initial_state = enc_st )
decoder_dense = keras.layers.Dense(len(italian_ch), activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)
print("#------------------------------4")
def dec_seq(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1, len(italian_ch)))
    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    target_seq[0, 0, tit_tindex["\t"]] = 1.0
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char
         # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_dec_it_len:
            stop_condition = True
        target_seq = np.zeros((1, 1, len(italian_ch)))
        target_seq[0, 0, sampled_token_index] = 1.0
        states_value = [h, c]
    return decoded_sentence
model_task1 = keras.Model([enc_inputs, dec_in], decoder_outputs)

model_task1.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

my_callbacks = [keras.callbacks.EarlyStopping(patience = 50)]
#Import load_model directly from keras.models to make it explicit.Moved the hyperparameters (batch_size, epochs, validation_split) to separate variables for betterreadability and easy modification.Used the load_model function to load the model, making the code more concise and readable
model_task1.fit([enc_indata, dec_indata], dec_tdata, batch_size = 64, epochs = 30, validation_split=0.2,)
model_task1.save("s2s")
model_task1 = keras.models.load_model("s2s")




enc_inputs = model_task1.input[0]
enc_outputs, state_h_enc, state_c_enc = model_task1.layers[2].output 
enc_st  = [state_h_enc, state_c_enc]
encoder_model = keras.Model(enc_inputs, enc_st )

dec_in = model_task1.input[1]
decoder_state_input_h = keras.Input(shape=(256,))
decoder_state_input_c = keras.Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model_task1.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(dec_in, initial_state=decoder_states_inputs)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model_task1.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model([dec_in] + decoder_states_inputs, [decoder_outputs] + decoder_states)

reverse_input_char_index = dict((i, char) for char, i in input_en_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in tit_tindex.items())


    
amount = 50
for seq_index in range(amount):
    input_seq = enc_indata[seq_index : seq_index + 1]
    decoded_sentence = dec_seq(input_seq)
    print("###################################################################")    
    print("The input sentence is : ", english_txt[seq_index])
    print("The resulted sentence is : ", decoded_sentence)
    print("###################################################################")

2023-12-29 22:45:22.793561: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-29 22:45:22.793654: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-29 22:45:22.866046: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-29 22:45:22.997364: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-29 22:45:24.820942: W tensorflow/compiler/tf2tens

#------------------------------0
#------------------------------1
English: It's over.
Italian: 	È finita.

English: I need a car.
Italian: 	Mi serve un'auto.

English: I had a job.
Italian: 	Io avevo un impiego.

English: We can talk.
Italian: 	Noi riusciamo a parlare.

English: Try this cake.
Italian: 	Provi questa torta.

#------------------------------2
#------------------------------3
#------------------------------4


2023-12-29 22:45:28.766506: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 226688000 exceeds 10% of free system memory.
2023-12-29 22:45:28.914797: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 226688000 exceeds 10% of free system memory.


Epoch 1/30
250/250 [==============================] - 48s 182ms/step - loss: 1.1757 - accuracy: 0.7055 - val_loss: 0.9593 - val_accuracy: 0.7242
Epoch 2/30
250/250 [==============================] - 47s 188ms/step - loss: 0.7631 - accuracy: 0.7744 - val_loss: 0.7892 - val_accuracy: 0.7617
Epoch 3/30
250/250 [==============================] - 47s 186ms/step - loss: 0.6549 - accuracy: 0.8002 - val_loss: 0.7007 - val_accuracy: 0.7854
Epoch 4/30
250/250 [==============================] - 44s 176ms/step - loss: 0.5971 - accuracy: 0.8154 - val_loss: 0.6526 - val_accuracy: 0.8003
Epoch 5/30
250/250 [==============================] - 45s 182ms/step - loss: 0.5555 - accuracy: 0.8264 - val_loss: 0.6205 - val_accuracy: 0.8081
Epoch 6/30
250/250 [==============================] - 42s 168ms/step - loss: 0.5236 - accuracy: 0.8353 - val_loss: 0.5994 - val_accuracy: 0.8137
Epoch 7/30
250/250 [==============================] - 45s 181ms/step - loss: 0.4946 - accuracy: 0.8442 - val_loss: 0.5809 - val_ac

INFO:tensorflow:Assets written to: s2s/assets


1/1 [==============================] - 0s 31ms/step
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
Input sentence: Hi.
Decoded sentence: Ciao.

^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
1/1 [==============================] - 0s 48ms/step
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
Input sentence: Hi.
Decoded sentence: Ciao.

^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
1/1 [==============================] - 0s 33ms/step
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
Input sentence: Run!
Decoded sentence: Corra!

^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
1/1 [==============================] - 0s 63ms/step
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
Input sentence: Run!
Decoded sentence: Corra!

^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
1/1 [==============================] - 0s 24ms/step
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
Input sentence: Run!
Decoded sentence: Corra!

^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
1/1 [==============================] - 0s 31ms/step
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
Input sentence: W